# Implémentation & Test de Dream-MCMC avec sous-échantillonnage

In [1]:
from core import *

## Tests

In [2]:
import pandas as pd
PARAM_LIST = ("moinslog10K", "n", "lambda_s", "rhos_cs")

capteur_riviere = pd.read_csv("../data/Point034_processed/processed_pressures.csv", sep = ',', names = ['dates', 'tension', 'temperature_riviere'], skiprows=1)
capteur_ZH = pd.read_csv("../data/Point034_processed/processed_temperatures.csv", sep = ',', names = ['dates', 'temperature_10', 'temperature_20', 'temperature_30', 'temperature_40'], skiprows=1)
etalonage_capteur_riv = pd.read_csv("../configuration/pressure_sensors/P508.csv")

def convertDates(df: pd.DataFrame):
    """
    Convert dates from a list of strings by testing several different input formats
    Try all date formats already encountered in data points
    If none of them is OK, try the generic way (None)
    If the generic way doesn't work, this method fails
    (in that case, you should add the new format to the list)
    
    This function works directly on the giving Pandas dataframe (in place)
    This function assumes that the first column of the given Pandas dataframe
    contains the dates as characters string type
    
    For datetime conversion performance, see:
    See https://stackoverflow.com/questions/40881876/python-pandas-convert-datetime-to-timestamp-effectively-through-dt-accessor
    """
    formats = ("%m/%d/%y %H:%M:%S", "%m/%d/%y %I:%M:%S %p",
               "%d/%m/%y %H:%M",    "%d/%m/%y %I:%M %p",
               "%m/%d/%Y %H:%M:%S", "%m/%d/%Y %I:%M:%S %p", 
               "%d/%m/%Y %H:%M",    "%d/%m/%Y %I:%M %p",
               "%y/%m/%d %H:%M:%S", "%y/%m/%d %I:%M:%S %p", 
               "%y/%m/%d %H:%M",    "%y/%m/%d %I:%M %p",
               "%Y/%m/%d %H:%M:%S", "%Y/%m/%d %I:%M:%S %p", 
               "%Y/%m/%d %H:%M",    "%Y/%m/%d %I:%M %p",
               None)
    times = df[df.columns[0]]
    for f in formats:
        try:
            # Convert strings to datetime objects
            new_times = pd.to_datetime(times, format=f)
            # Convert datetime series to numpy array of integers (timestamps)
            new_ts = new_times.values.astype(np.int64)
            # If times are not ordered, this is not the appropriate format
            test = np.sort(new_ts) - new_ts
            if np.sum(abs(test)) != 0 :
                #print("Order is not the same")
                raise ValueError()
            # Else, the conversion is a success
            #print("Found format ", f)
            df[df.columns[0]] = new_times
            return
        
        except ValueError:
            #print("Format ", f, " not valid")
            continue
    
    # None of the known format are valid
    raise ValueError("Cannot convert dates: No known formats match your data!")

convertDates(capteur_riviere)
convertDates(capteur_ZH)

# set seed for reproducibility
np.random.seed(0)

# conversion des mesures de pression
intercept = float(etalonage_capteur_riv['P508'][2])
a = float(etalonage_capteur_riv['P508'][3])
b = float(etalonage_capteur_riv['P508'][4])
capteur_riviere['dH'] = (capteur_riviere['tension'].astype(float)-intercept-capteur_riviere['temperature_riviere'].astype(float)*b)/a

# conversion mesures de tempétratures
capteur_riviere['temperature_riviere'] = capteur_riviere['temperature_riviere'] + 273.15
capteur_ZH['temperature_10'] = capteur_ZH['temperature_10'] + 273.15
capteur_ZH['temperature_20'] = capteur_ZH['temperature_20'] + 273.15
capteur_ZH['temperature_30'] = capteur_ZH['temperature_30'] + 273.15
capteur_ZH['temperature_40'] = capteur_ZH['temperature_40'] + 273.15

# définition des attributs de colonnes
dH_measures = list(zip(capteur_riviere['dates'],list(zip(capteur_riviere['dH'], capteur_riviere['temperature_riviere']))))
T_measures = list(zip(capteur_ZH['dates'], capteur_ZH[['temperature_10', 'temperature_20', 'temperature_30', 'temperature_40']].to_numpy()))

col_dict = {
	"river_bed": 1., 
    "depth_sensors": [.1, .2, .3, .4],
	"offset": .0,
    "dH_measures": dH_measures,
	"T_measures": T_measures,
    "sigma_meas_P": None,
    "sigma_meas_T": None,
    "inter_mode": 'lagrange'
}

col = Column.from_dict(col_dict)

params = Param(
    moinslog10K = 7,
    n = .1,
    lambda_s = 2,
    rhos_cs = 4e6
)

params_tuple = (7, .1, 2, 4e6)

assert params == params_tuple

col.compute_solve_transi(params, nb_cells=100)

--- Compute Solve Transi ---
Layer 1 : ends at 0.4 m. Parametres(moinslog10K=7, n=0.1, lambda_s=2, rhos_cs=4000000.0)
Done.


In [3]:
priors_couche_1 = {
    "moinslog10K": ((1, 10), .01), # (intervalle, sigma)
    "n": ((.001, .25), .005),
    "lambda_s": ((1, 10), .1),
    "rhos_cs": ((1e6,1e7), 1e5),
}

priors_couche_2 = {
    "moinslog10K": ((4, 9), .01), # (intervalle, sigma)
    "n": ((.001, .25), .005),
    "lambda_s": ((1, 10), .1),
    "rhos_cs": ((1e6,1e7), 1e5),
}

priors_couche_3 = {
    "moinslog10K": ((4, 9), .01), # (intervalle, sigma)
    "n": ((.001, .25), .005),
    "lambda_s": ((1, 10), .1),
    "rhos_cs": ((1e6,1e7), 1e5),
}

Exécution de dream_mcmc sur l'objet Column

In [6]:
nb_chain = 10
nb_iter = 1000
nb_param = 4
nb_cells = 100

all_priors = [
    ['Couche 1', 0.4, priors_couche_1]
]

if not isinstance(all_priors, AllPriors):
    all_priors = AllPriors([LayerPriors(*conv(layer)) for layer in all_priors])

col.compute_dream_mcmc_without_sigma2(
    nb_iter,
    all_priors,
    nb_cells,
    nb_chain=nb_chain,
    verbose=True
)
process = psutil.Process()
print(f"Après la fin de l'exécution - Utilisation de la mémoire (en Mo) : {process.memory_info().rss /1e6}")

--- Compute DREAM MCMC ---
Priors :
    [Prior sur une valeure qui évolue entre 1 et 10, Prior sur une valeure qui évolue entre 0.001 et 0.25, Prior sur une valeure qui évolue entre 1 et 10, Prior sur une valeure qui évolue entre 1000000.0 et 10000000.0]
Number of cells : 100
Number of iterations : 1000
Number of chains : 10
--------------------
Initialisation - Utilisation de la mémoire (en Mo) : 287.698944
--- Begin Burn in phase ---
Burn in finished after : 101 iterations
Initialisation post burn-in - Utilisation de la mémoire (en Mo) : 332.111872
DREAM MCMC Computation: 100%|██████████| 1000/1000 [02:00<00:00,  8.29it/s]
Occupation mémoire des températures (en Mo) :  528.836
Occupation mémoire des flux (en Mo) :  528.836
Fin itérations MCMC, avant le calcul des quantiles - Utilisation de la mémoire (en Mo) : 1379.438592
Quantiles computed
Fin de l'exécution - Utilisation de la mémoire (en Mo) : 1385.795584
Après la fin de l'exécution - Utilisation de la mémoire (en Mo) : 307.163136

## Histogrammes

In [4]:
_params = np.array([col._states[k].layers[0].params for k in range(len(col._states))])
threshold = 1.1

plt.figure(figsize=(15,15))
plt.title(f"Threshold = {threshold}        Burn-in over after {col.nb_burn_in_iter} iterations")
plt.axis('off')
dico = {0 : "$-log(K)$", 1 : "$\lambda_s$", 2 : "$n$", 3 : "$rho * C$"}
for k in range(nb_param):
    plt.subplot(2, 2, k+1)
    plt.hist(_params[:,k], bins = 100, label=f"{dico[k]}")
    plt.legend();
plt.savefig(f"../images/threshold_{int(threshold*100)}.png")

TypeError: object of type 'NoneType' has no len()

## Estimation de la mémoire vive nécessaire pour l'inversion MCMC

Deux approches possibles pour l'utilisateur :
* L'ordinateur lui propose un pas de sous-échantillonnage directement. 10 dans le meilleur des cas, plus sinon.
* Il entre ses pas de sous-échantillonnage puis l'ordinateur lui dit si ça passe au niveau de la mémoire vive;

Nous allons laisser les deux possibilités à l'utilisateur. Il pourra par exemple cliquer sur un bouton "pas de sous-échantillonnage automatique" pour activer la seconde option, sinon il entrera lui-même les pas de sous-échantillonnage et on utilisera le code de l'option 1.

### Estimation de la mémoire vive impliquée

In [31]:
def ram_estimation(nb_iter, nb_chain, nb_cells, nb_times, nb_layer, n_sous_ech_iter=10, n_sous_ech_space=1, n_sous_ech_time=1, nb_bytes=4):
    """
    Input : le nombre d'itérations, de chaînes, de cellules, de couches,
            les pas de sous-échantillonnage respectivement pour les itérations MCMC,
            l'espace et le temps
            nb_bytes est la taille en octets d'un élément de tableau, par exemple 4 pour float32
    Output : estimation de la mémoire vive recquise pour réaliser les inversions MCMC
    Method : calcule la RAM nécessaire pour stocker les arrays et y ajoute x % (à déterminer)
    """
    nb_iter_sous_ech = int( np.ceil( (nb_iter+1) / n_sous_ech_iter))
    nb_cells_sous_ech = int( np.ceil(nb_cells / n_sous_ech_space) )
    nb_times_sous_ech = int( np.ceil(nb_times / n_sous_ech_time) )



    ram = 0 # initialisation
    ram += (4*nb_chain+1)*nb_cells*nb_times # _temp_act, temp_old, temp_new, _flow_act, _flow_old
    ram += 2*nb_chain # _energy, _energy_old
    ram += nb_cells*nb_times # temp_new
    ram += nb_iter*nb_chain # acceptance
    ram += (nb_iter+1)*nb_chain*nb_layer*nb_param # _params
    ram += (nb_chain+2)*nb_layer*nb_param #x_new, X_new et dX
    ram += 2*nb_iter_sous_ech*nb_chain*nb_cells_sous_ech*nb_times_sous_ech # _flows, _temp
    ram += 6*nb_cells_sous_ech*nb_times_sous_ech # quantiles_temps, quantiles_flows

    ram *= 1.3 # Correction

    return ram*nb_bytes # + inconnue à déterminer dans la partie suivante

In [32]:
nb_chain = 20
nb_iter = 500
nb_param = 4
nb_cells = 100
nb_times = 1309
nb_layer = 1

n_sous_ech_iter = 10
n_sous_ech_space = 1
n_sous_ech_time = 1

# ram_est = ram_estimation(nb_iter, nb_chain, nb_cells, nb_times, nb_layer, )
# print("Estimation de la mémoire recquise (en Mo) : ", ram_est /1e6)

# all_priors = [
#     ['Couche 1', 0.4, priors_couche_1]
# ]

# if not isinstance(all_priors, AllPriors):
#     all_priors = AllPriors([LayerPriors(*conv(layer)) for layer in all_priors])

# col.compute_dream_mcmc_without_sigma2(
#     nb_iter,
#     all_priors,
#     nb_cells,
#     n_sous_ech_iter=n_sous_ech_iter,
#     n_sous_ech_space=n_sous_ech_space,
#     n_sous_ech_time=n_sous_ech_time,
#     nb_chain=nb_chain,
#     verbose=True
# )
# process = psutil.Process()
# print(f"Après la fin de l'exécution - Utilisation de la mémoire (en Mo) : {process.memory_info().rss /1e6}")

### Option 1 : l'ordinateur propose un pas de sous-échantillonnage

In [35]:
def propose_n_iter_sous_ech(nb_iter, nb_chain, nb_cells, nb_times, nb_layer, n_sous_ech_space=1, n_sous_ech_time=1, nb_bytes=4):
    n_sous_ech_iter = 10
    while ram_estimation(nb_iter, nb_chain, nb_cells, nb_times, nb_layer, n_sous_ech_iter, n_sous_ech_space, n_sous_ech_time, nb_bytes) > 2e9:
        n_sous_ech_iter += 1
    print(f"You should use n_sous_ech_iter = ", n_sous_ech_iter)
    return n_sous_ech_iter

### Option 2 : l'utilisateur entre lui-même les pas de sous-échantillonnage

Avertissement en cas de dépassement du seuil de mémoire vive (2Go) et confirmation de continuation. En cas de refus, proposer un pas de sous-échantillonnage en itérations MCMC possible en conservant les autres pas de sous-échantillonnage.

In [27]:
def warning_ram(nb_iter, nb_chain, nb_cells, nb_times, nb_layer, n_sous_ech_iter=10, n_sous_ech_space=1, n_sous_ech_time=1, nb_bytes=4):
    """
    Input : Parameters of MCMC
    Output : Bool - Do we compute MCMC ?
             Parameters - if bool, parameters used in the MCMC we are about to compute.
    """
    ram_est = ram_estimation(nb_iter, nb_chain, nb_cells, nb_times, nb_layer, n_sous_ech_iter, n_sous_ech_space, n_sous_ech_time, nb_bytes)
    if ram_est > 2e9:
        while True:
            user_input = input(f"// WARNING - The given parameters recquire {ram_est/1e9} > 2Go RAM for the MCMC inversions \\ \n Input 0 if you want to continue anyway \n Input 1 if you want the computer to automatically choose the parameters in order to be under 2Go RAM \n Input 2 if you want to stop the algorithm.")
            print(f"// WARNING - The given parameters recquire {ram_est/1e9} > 2Go RAM for the MCMC inversions \n Input 0 if you want to continue anyway \n Input 1 if you want the computer to automatically choose the parameters in order to be under 2Go RAM \n Input 2 if you want to stop the algorithm.")

            # Vérifie si l'entrée est valide
            if user_input in ('0', '1', '2'):
                break  # Sort de la boucle si l'entrée est valide
            else:
                print("Entrée invalide. Veuillez entrer 0; 1 ou 2.")

        if user_input == 0:
            # Continue anyway
            return True, [nb_iter, nb_chain, nb_cells, nb_times, nb_layer, n_sous_ech_iter, n_sous_ech_space, n_sous_ech_time, nb_bytes]
        
        elif user_input == 1:
            # Compute parameters in order to use less than 2Go RAM for the MCMC inversions.
            parameters = ...
            return True, parameters
        
        else:
            # Stop the algorithm
            return False, []
        
    else:
        return True, [nb_iter, nb_chain, nb_cells, nb_times, nb_layer, n_sous_ech_iter, n_sous_ech_space, n_sous_ech_time, nb_bytes]


In [28]:
nb_iter = 30000
warning_ram(nb_iter, nb_chain, nb_cells, nb_times, nb_layer, n_sous_ech_iter, n_sous_ech_space, n_sous_ech_time)

// WARNING - The given parameters recquire 81.78432812160001 > 2Go RAM for the MCMC inversions \ 
 Input 0 if you want to continue anyway 
 Input 1 if you want the computer to automatically choose the parameters in order to be under 2Go RAM 
 Input 2 if you want to stop the algorithm.


(False, [])